<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Waldo Campos Maldonado


### **Link de repositorio de GitHub:** `https://github.com/WaldoCampos/mds7202`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
# !pip install praw
# !pip install line_profiler
# !pip install numba
# !pip install memory_profiler

%load_ext autoreload
%autoreload 2
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [2]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [3]:
votes, post, url = praw_reddit()

In [4]:
%load_ext line_profiler
%lprun -f praw_reddit praw_reddit()

In [5]:
%load_ext memory_profiler
%memit praw_reddit()

peak memory: 149.81 MiB, increment: 6.92 MiB


In [6]:
%%file bench_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting bench_module.py


In [7]:
from bench_module import praw_reddit

%mprun -f praw_reddit praw_reddit()

**Respuesta:**

Solo la definición de la función y la función que carga todos los posts y su información ocupan una cantidad significativa de memoria, el resto de operaciones gasta muy poco en comparación.

La parte más lenta de la función es la que carga toda la información de los posts de reddit en una lista, es más de 1000 veces más lenta que el resto de operaciones.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

1. La memoria cache es un espacio donde se almacenan datos cuya obtención suele ser demorosa, por lo general la memoria cache suele ser memoria de rápido acceso y baja capacidad. Las siglas LRU significan "least recently used", es decir, usada menos frecuente, esto significa que cuando la memoria cache llega a su límite y se necesitan borrar elementos para poder seguir introduciendo nuevos, entonces los elementos usados menos frecuentemente son los eliminados.

2. El aplicar caching aumenta considerablemente el uso de memoria de los programas.

3. Si se aplica caching sobre la función anterior entonces solo la primera ejecución demoraría bastante, las siguientes ejecuciones solo demorarán el tiempo que toma leer el cache, siempre y cuando se ejecute con los mismos argumentos.

In [8]:
from functools import lru_cache

@lru_cache
def praw_reddit_2(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [9]:
start = time.time()
a = praw_reddit()
print(f"Tiempo total función sin cache: {time.time() - start}")

Tiempo total función sin cache: 16.415542602539062


In [10]:
start = time.time()
b = praw_reddit_2()
print(f"Tiempo total función con cache primera ejecución: {time.time() - start}")

Tiempo total función con cache primera ejecución: 16.172444105148315


In [11]:
start = time.time()
c = praw_reddit_2()
print(f"Tiempo total función con cache segunda ejecución: {time.time() - start}")

Tiempo total función con cache segunda ejecución: 0.0


**Respuesta de lru_cache sobre praw_reddit**

La segunda ejecución de la función es casi instantánea mientras que la primera ejecución demora casi lo mismo que sin el decorador de LRU cache.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [12]:
def intervals(upvotes, downvotes):
    sigma = []
    mu = []
    for u, d in zip(upvotes, downvotes):
        a = 1 + u
        b = 1 + d
        sigma.append(1.65 * (((a*b) / (((a+b)**2)*(a+b+1)))**(0.5)))
        mu.append(a / (a+b))
    return mu, sigma

def intervals_numpy(upvotes, downvotes):
    a = 1 + upvotes
    b = 1 + downvotes
    sigma = 1.65 * np.sqrt((a*b) / (((a+b)**2)*(a+b+1)))
    mu = a / (a+b)
    return mu, sigma

In [13]:
intervals(votes[0][:, 0], votes[0][:, 1])

([0.8142857142857143,
  0.6666666666666666,
  0.9878971255673222,
  0.967479674796748,
  0.9975247524752475,
  0.9655172413793104,
  0.9770491803278688,
  0.7575757575757576,
  0.9263157894736842,
  0.8367346938775511,
  0.7674418604651163,
  0.9857142857142858,
  0.9318181818181818,
  0.7862595419847328,
  0.937125748502994,
  0.9195402298850575,
  0.9125,
  0.9545454545454546,
  0.825,
  0.8933333333333333,
  0.9950248756218906,
  0.7543859649122807,
  0.8181818181818182,
  0.8333333333333334,
  0.8823529411764706,
  0.8421052631578947,
  0.85,
  0.9333333333333333,
  0.9204545454545454,
  0.9629629629629629,
  0.9790419161676647,
  0.8,
  0.9180327868852459,
  0.6428571428571429,
  0.84,
  0.7631578947368421,
  0.7575757575757576,
  0.7368421052631579,
  0.9285714285714286,
  0.864406779661017,
  0.8,
  0.8333333333333334,
  0.7160493827160493,
  0.85,
  0.7272727272727273,
  0.8284518828451883,
  0.8892773892773893,
  0.7777777777777778,
  0.625,
  0.8431372549019608],
 [0.07614928

In [14]:
intervals_numpy(votes[0][:, 0], votes[0][:, 1])

(array([0.81428571, 0.66666667, 0.98789713, 0.96747967, 0.99752475,
        0.96551724, 0.97704918, 0.75757576, 0.92631579, 0.83673469,
        0.76744186, 0.98571429, 0.93181818, 0.78625954, 0.93712575,
        0.91954023, 0.9125    , 0.95454545, 0.825     , 0.89333333,
        0.99502488, 0.75438596, 0.81818182, 0.83333333, 0.88235294,
        0.84210526, 0.85      , 0.93333333, 0.92045455, 0.96296296,
        0.97904192, 0.8       , 0.91803279, 0.64285714, 0.84      ,
        0.76315789, 0.75757576, 0.73684211, 0.92857143, 0.86440678,
        0.8       , 0.83333333, 0.71604938, 0.85      , 0.72727273,
        0.82845188, 0.88927739, 0.77777778, 0.625     , 0.84313725]),
 array([0.07614929, 0.29398737, 0.00701221, 0.01521533, 0.00407406,
        0.02107899, 0.01412474, 0.0409616 , 0.04399619, 0.05012956,
        0.05299677, 0.02323707, 0.03606268, 0.05887397, 0.0218825 ,
        0.04784289, 0.05180383, 0.05123475, 0.09791247, 0.05842494,
        0.00816822, 0.09325955, 0.07774816, 0.

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [15]:
from numba import jit


@jit(nopython=True)
def intervals_numba(upvotes, downvotes):
    sigma = []
    mu = []
    for u, d in zip(upvotes, downvotes):
        a = 1 + u
        b = 1 + d
        sigma.append(1.65 * (((a*b) / (((a+b)**2)*(a+b+1)))**(0.5)))
        mu.append(a / (a+b))
    return mu, sigma

@jit(nopython=True)
def intervals_numpy_numba(upvotes, downvotes):
    a = 1 + upvotes
    b = 1 + downvotes
    sigma = 1.65 * np.sqrt((a*b) / (((a+b)**2)*(a+b+1)))
    mu = a / (a+b)
    return mu, sigma

In [17]:
it = []
tiempos_python = []
tiempos_numpy = []
tiempos_python_numba = []
tiempos_numpy_numba = []
for key in votes:
    it.append(len(votes[key]))
    
    k = 1000

    start = time.time()
    for i in range(k):
        a1, a2 = intervals(votes[key][:, 0], votes[key][:, 1])
    end = time.time()
    tiempos_python.append((end - start)/k)
    
    start = time.time()
    for i in range(k):
        b1, b2 = intervals_numpy(votes[key][:, 0], votes[key][:, 1])
    end = time.time()
    tiempos_numpy.append((end - start)/k)
    
    start = time.time()
    for i in range(k):
        c1, c2 = intervals_numba(votes[key][:, 0], votes[key][:, 1])
    end = time.time()
    tiempos_python_numba.append((end - start)/k)
    
    start = time.time()
    for i in range(k):
        d1, d2 = intervals_numpy_numba(votes[key][:, 0], votes[key][:, 1])
    end = time.time()
    tiempos_numpy_numba.append((end - start)/k)

df = pd.DataFrame(data={'it': it,
                        'Python': tiempos_python,
                        'Numpy': tiempos_numpy,
                        'Python-JIT': tiempos_python_numba,
                        'Numpy-JIT': tiempos_numpy_numba})

<ipython-input-12-e0e307daa24e>:7: RuntimeWarning: overflow encountered in long_scalars
  sigma.append(1.65 * (((a*b) / (((a+b)**2)*(a+b+1)))**(0.5)))
<ipython-input-12-e0e307daa24e>:7: RuntimeWarning: invalid value encountered in double_scalars
  sigma.append(1.65 * (((a*b) / (((a+b)**2)*(a+b+1)))**(0.5)))
<ipython-input-12-e0e307daa24e>:14: RuntimeWarning: invalid value encountered in sqrt
  sigma = 1.65 * np.sqrt((a*b) / (((a+b)**2)*(a+b+1)))


In [19]:
df2 = pd.melt(df, id_vars=['it'], var_name='tipo', value_name='Tiempo')

fig = px.line(df2, x='it', y='Tiempo', color='tipo')
fig.write_html("./mediciones_output.html")
fig.show()

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Respuesta:** Solo se puede observar una diferencia significativa entre las ejecuciones con python base y las demás funciones, esta diferencia puede deberse a que tanto numpy como numba trabajan por debajo con un lenguaje de menor nivel y, en el caso de numba, optimizan la ejecución de las funciones.

El resultado más rápido resulta ser Numpy con JIT, este se demora cerca de un 25% de lo que demora Numpy. Mientras que Python con JIT cerca del doble de lo que demora Numpy.

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [20]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervals_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.write_html("./posts_output.html")
fig.show()

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



<ipython-input-12-e0e307daa24e>:14: RuntimeWarning:

invalid value encountered in sqrt



**Respuesta:**

Todos los posts en el top 20 resultan tener una media muy cercana a 1, esto quiere decir que es muy altamente probable que sean buenos posts. Esto quiere decir que los posts que obtienen la mayor cantidad de votos en el sitio es muy altamente probable que sean buenos de acuerdo al posterior.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>